# Kredi Kartı Dolandırıcılığı Tespiti

Önceki egzersizler, bir sinir ağının tüm farklı bileşenlerine daha yakından bakmanızı sağladı:
* sıralı (sequential) yoğun (Dense) bir Sinir Ağı mimarisi,
* derleme (compilation) yöntemi,
* modelin eğitilmesi (fitting).

Şimdi **çok fazla veri içeren** gerçek hayat bir veri seti üzerinde çalışalım!

**Veri seti: `Kredi Kartı İşlemleri (Credit Card Transactions)`**

Bu açık uçlu challenge’da, **kredi kartı işlemlerinden elde edilmiş verilerle** çalışacaksınız.

Bu veriler **hassas** olduğu için, toplam 31 sütundan yalnızca 3’ü bilinmektedir; geri kalanlar verileri **anonimleştirmek** amacıyla dönüştürülmüştür (aslında bunlar, orijinal verilerin **PCA projeksiyonlarıdır**).

Bilinen 3 sütun şunlardır:

* `TIME`: İşlemin, veri setindeki ilk işleme göre geçen süresi  
* `AMOUNT`: İşlem tutarı  
* `CLASS` (hedef değişkenimiz):
    * `0 : geçerli işlem`
    * `1 : sahte (fraud) işlem`

❓ **Soru** ❓ Veri setini indirerek başlayın:
* Kaggle üzerinden: [buradan](https://www.kaggle.com/mlg-ulb/creditcardfraud)
* veya bizim URL’mizden: [buradan](https://d32aokrjazspmn.cloudfront.net/materials/creditcard.csv)

Veriyi yükleyerek `X` ve `y` değişkenlerini oluşturun.

In [1]:
import pandas as pd

# Veriyi verilen linkten doğrudan çekiyoruz
url = "https://d32aokrjazspmn.cloudfront.net/materials/creditcard.csv"
df = pd.read_csv(url)

# Veriyi X ve y olarak ayıralım
X = df.drop(columns=['Class'])  # Cevap anahtarını (Class) çıkar, geriye kalanlar ipuçlarıdır
y = df['Class']                 # Sadece cevap anahtarını al

# Verinin boyutlarına bakalım (Satır, Sütun)
print(f"X boyutu: {X.shape}")
print(f"y boyutu: {y.shape}")# SENİN KODUN BURAYA

X boyutu: (284807, 30)
y boyutu: (284807,)


## 1. Sınıfların yeniden dengelenmesi

In [2]:
# Sınıf dengesini kontrol edelim
pd.Series(y).value_counts(normalize=True)

Class
0    0.998273
1    0.001727
Name: proportion, dtype: float64

☝️ Bu `fraud detection` (sahtekârlık tespiti) challenge’ında **sınıflar aşırı derecede dengesizdir**:
* %99.8 normal işlemler
* %0.2 sahte (fraud) işlemler

**Ciddi yeniden dengeleme (rebalancing) stratejileri uygulamazsak sahtekârlık vakalarını tespit edemeyiz!**

❓ **Soru** ❓
1. **Önce**, veri setinizden üç ayrı bölme oluşturun: `Train / Val / Test`.  
   Doğrulama (validation) ve test setlerinin **dengesiz** kalması son derece önemlidir; böylece modeli değerlendirirken gerçek koşullar korunur ve veri sızıntısı (data leakage) oluşmaz. Test setinizi bu notebook’un **en son hücresine kadar saklayın**!

&nbsp;

2. **İkinci olarak**, yalnızca eğitim setinizi (training set) yeniden dengeleyin. Birçok seçeneğiniz var:

- Azınlık sınıfını rastgele oversample etmek (düz NumPy fonksiyonlarıyla).  
  *(En iyi seçenek değildir; çünkü satırları kopyalayarak veri sızıntısı yaratır.)*
- <a href="https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/">**`Synthetic Minority Oversampling Technique - SMOTE`**</a> kullanarak, mevcut gözlemleri ağırlıklandırıp yeni veri noktaları üretmek
- Buna ek olarak, çoğunluk sınıfını bir miktar küçültmek için  
  <a href="https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/">**`RandomUnderSampler`**</a> da deneyebilirsiniz

In [3]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# 1. ADIM: Veriyi Ayırma (Train / Val / Test)
# Önce Test setini ayıralım (Kenara koyuyoruz, dokunmayacağız)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Şimdi elimizde kalan parçayı Train ve Validation olarak ayıralım
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=42, stratify=y_train_val)

# Kontrol edelim: Şu an her şey hala orjinal ve dengesiz mi?
print(f"Orijinal Train seti boyutu: {X_train.shape}")
print(f"Orijinal Train içindeki dolandırıcı oranı: {sum(y_train)/len(y_train):.4f}")


# 2. ADIM: SADECE Eğitim Setini Dengeleme (SMOTE)
# Teraziyi dengeliyoruz: Azınlık sınıfı (dolandırıcıları) sentetik olarak çoğaltıyoruz.
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Sonuçları görelim
print("\n--- SMOTE Sonrası ---")
print(f"Yeni Train seti boyutu: {X_train_smote.shape}")
print(f"Yeni Train içindeki dolandırıcı sayısı: {sum(y_train_smote)}")
print(f"Yeni Train içindeki normal işlem sayısı: {len(y_train_smote) - sum(y_train_smote)}")

Orijinal Train seti boyutu: (139554, 30)
Orijinal Train içindeki dolandırıcı oranı: 0.0017

--- SMOTE Sonrası ---
Yeni Train seti boyutu: (278626, 30)
Yeni Train içindeki dolandırıcı sayısı: 139313
Yeni Train içindeki normal işlem sayısı: 139313


## 2. Sinir Ağı yinelemeleri

Sınıflarınızı yeniden dengelediğinize göre, test puanınızı optimize etmek için bir sinir ağı uyarlayın. Aşağıdaki ipuçlarını kullanmaktan çekinmeyin:

- Girişlerinizi normalleştirin!
    - Modelinizdeki ön işlemeyi “boru hattı” haline getirmek için tercihen model içinde bir [`Normalization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization) katmanı kullanın. 
    - Veya modelinizin dışında sklearn'in [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) öğesini kullanın, `X_train`, `X_val` ve `X_test` öğelerini uygulayın.
- Modelinizi aşırı uyumlu hale getirin, ardından aşağıdakileri kullanarak düzenleyin:
- Erken Durdurma kriterleri
- [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) katmanları
    - veya [`düzenleyiciler`](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers) katmanları
- 🚨 İzlemek istediğiniz metrikleri ve kullanmak istediğiniz kayıp fonksiyonunu dikkatlice düşünün!

In [4]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import models, layers, metrics, callbacks

# --- 1. NORMALİZASYON (SCALING) ---
scaler = StandardScaler()

# Scaler'ı sadece EĞİTİM setine göre ayarla (fit)
X_train_scaled = scaler.fit_transform(X_train_smote)

# Aynı ayarı Val ve Test setlerine uygula (transform)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# --- 2. MODEL MİMARİSİ ---
model = models.Sequential()

# Giriş Katmanı + İlk Gizli Katman
# input_dim = Sütun sayısı (30 civarı)
model.add(layers.Dense(32, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(layers.Dropout(0.5)) # Nöronların %50'sini rastgele kapat (Ezberlemeyi önler)

# İkinci Gizli Katman
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))

# Çıkış Katmanı
# Dolandırıcı mı (1) değil mi (0)? Tek bir nöron ve Sigmoid yeterli.
model.add(layers.Dense(1, activation='sigmoid'))

# --- 3. DERLEME (COMPILE) ---
# Metrics kısmına dikkat! Sadece accuracy değil, Recall ve Precision da ekliyoruz.
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', metrics.Recall(name='recall'), metrics.Precision(name='precision')])

# --- 4. EĞİTİM (FIT) ---
# EarlyStopping: Eğer 'val_loss' 10 epoch boyunca iyileşmezse eğitimi durdur.
es = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train_smote,
                    validation_data=(X_val_scaled, y_val),
                    batch_size=64, # Her seferde 64 veri al
                    epochs=50,     # En fazla 50 tur dön (EarlyStopping muhtemelen önce durdurur)
                    callbacks=[es],
                    verbose=1)

2026-01-21 23:50:16.688999: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-21 23:50:16.701757: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-21 23:50:16.800841: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-21 23:50:16.929915: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-21 23:50:17.045373: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

Epoch 1/50


/home/tumay/.pyenv/versions/workintech/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2026-01-21 23:50:20.427221: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2026-01-21 23:50:20.429001: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


4354/4354 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9544 - loss: 0.1290 - precision: 0.9655 - recall: 0.9425 - val_accuracy: 0.9925 - val_loss: 0.0314 - val_precision: 0.1770 - val_recall: 0.9126
Epoch 2/50
4354/4354 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9820 - loss: 0.0556 - precision: 0.9836 - recall: 0.9804 - val_accuracy: 0.9952 - val_loss: 0.0233 - val_precision: 0.2520 - val_recall: 0.9126
Epoch 3/50
4354/4354 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9862 - loss: 0.0424 - precision: 0.9851 - recall: 0.9873 - val_accuracy: 0.9963 - val_loss: 0.0191 - val_precision: 0.3056 - val_recall: 0.8932
Epoch 4/50
4354/4354 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9880 - loss: 0.0369 - precision: 0.9859 - recall: 0.9901 - val_accuracy: 0.9962 - val_loss: 0.0188 - val_precision: 0.2987 - val_recall: 0.8932
Epoch 5/50
4354/4354 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9885 - loss: 0.0353 - precision: 0.9859 - recall: 0.9911 - val_accuracy: 0.9966 - val_loss: 0.0

### 🧪 Kodunu Test Et

Orijinal dengesiz veri kümesinin (`X_test`, `y_test`) temsil edici bir örneğinde gerçek test performansınızın altında kalan sonuçları `precision` ve `recall` değişkenlerine kaydedin.

In [5]:
from sklearn.metrics import precision_score, recall_score

# 1. Modelden tahminleri al (Olasılık olarak döner: örn 0.85, 0.12)
y_pred_probs = model.predict(X_test_scaled)

# 2. Olasılıkları 0 veya 1'e çevir (Eşik değerimiz 0.5)
# %50'den büyükse Dolandırıcı (1), küçükse Normal (0) diyelim
y_pred = (y_pred_probs > 0.5).astype(int)

# 3. Skorları hesapla
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Precision (Kesinlik): {precision:.4f}")
print(f"Recall (Duyarlılık): {recall:.4f}")

print("\n--- Yorum ---")
print(f"Tespit edilen dolandırıcı oranı (Recall): %{recall*100:.2f}")
print(f"Suçladıklarımızdan gerçekten dolandırıcı olanlar (Precision): %{precision*100:.2f}")

2671/2671 ━━━━━━━━━━━━━━━━━━━━ 1s 507us/step
Precision (Kesinlik): 0.3708
Recall (Duyarlılık): 0.8243

--- Yorum ---
Tespit edilen dolandırıcı oranı (Recall): %82.43
Suçladıklarımızdan gerçekten dolandırıcı olanlar (Precision): %37.08


In [6]:
from nbresult import ChallengeResult

result = ChallengeResult('solution',
    precision=precision,
    recall=recall,
    fraud_number=len(y_test[y_test == 1]),
    non_fraud_number=len(y_test[y_test == 0]),
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/tumay/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/tumay/code/S18D3-S-Data-credit-card-challenge/tests
plugins: anyio-4.8.0, dash-3.3.0, typeguard-4.4.2
collecting ... collected 2 items

test_solution.py::TestSolution::test_is_score_good_enough PASSED         [ 50%]
test_solution.py::TestSolution::test_is_test_set_representative PASSED   [100%]

============================== 2 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/solution.pickle

git commit -m 'Completed solution step'

git push origin master



## 🏁 İsteğe bağlı: Bu zorluk için Google'ın çözümünü okuyun
Bu oturumdaki tüm zorlukları tamamladığınız için tebrikler!

Son olarak, Google'ın kendi çözümünü doğrudan [Colab'da buradan](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb) okuyun. 

İlginç teknikler ve en iyi uygulamaları keşfedeceksiniz.